In [3]:
import pandas as pd
import time
import random


from bs4 import BeautifulSoup as bs
import requests
import numpy as np

from IPython.display import display
from datetime import datetime

from dotenv import load_dotenv
import os
load_dotenv()




alimentacion = {
    "Leche Huevos Yogures y Lácteos": "https://www.alcampo.es/compra-online/alimentacion/leche-huevos-yogures-y-lacteos/c/W16?q=%3Arelevance&page=",
    "Aceite Vinagre Salsas Especias" : "https://www.alcampo.es/compra-online/alimentacion/aceite-vinagre-salsas-especias/c/W18?q=%3Arelevance&page=",
    "Desayuno y Merienda": "https://www.alcampo.es/compra-online/alimentacion/desayuno-y-merienda/c/W10?q=%3Arelevance&page=",
    # "Conservas": "https://www.alcampo.es/compra-online/alimentacion/conservas/c/W1004?q=%3Arelevance&page=",
    # "Comida Internacional":"https://www.alcampo.es/compra-online/alimentacion/comida-internacional/c/W9410?q=%3Arelevance&page=",
    # "Aperitivos, aceitunas y frutos secos": "https://www.alcampo.es/compra-online/alimentacion/aperitivos-aceitunas-y-frutos-secos/c/W120?q=%3Arelevance&page=",
    # "Nutrición deportiva":"https://www.alcampo.es/compra-online/alimentacion/nutricion-deportiva/c/W12?q=%3Arelevance&page=",
    # "Caldos, Pasta, Arroz Legumbres Puré":"https://www.alcampo.es/compra-online/alimentacion/caldos-pasta-arroz-legumbres-pure/c/W140?q=%3Arelevance&page=",
    # "Azúcar Harina Masas y Pan": "https://www.alcampo.es/compra-online/alimentacion/azucar-harina-masas-y-pan/c/W1009?q=%3Arelevance&page=",
    # "Alimentos Especiales": "https://www.alcampo.es/compra-online/alimentacion/alimentos-especiales-/c/W33?q=%3Arelevance&page=",
    # "Veganos": "https://www.alcampo.es/compra-online/alimentacion/veganos/c/W09112021?q=%3Arelevance&page=",
    # "Supermercado Ecológico": "https://www.alcampo.es/compra-online/alimentacion/supermercado-ecologico/c/W26112021?q=%3Arelevance&page="
}



dict_alcampo = {"Category": [], "Name":[],"Price":[],"Stock":[], "Image":[]}

for i in alimentacion.items():

    categoria = i[0]
    url_to_scrape = i[1]
    contador = 0
    product_not_found = 10

    while product_not_found != "0":
        
        url = url_to_scrape+ f"{contador}"
        response = requests.get(url='https://proxy.scrapeops.io/v1/',params={'api_key': os.environ["SCARPE_OPS_API_KEY"],'url': url, },)
        html = response.content
        soup = bs(html,"lxml")
     
        print(categoria,url)
        try:
            product_not_found = soup.find("div",class_="totalResults").get_text(strip=True, separator='').split(" ")[0]
        except:
            print("Got an attribute error on:",url)

        if product_not_found =="0":
            print("No hay más ventanas")
            break
        
        else:
            products = soup.find_all("div",class_="productGridItemContainer")
            prices = soup.find_all("div",class_="priceContainer")
            stock_available = soup.find_all("div",class_="cart add")

            for product in products:

                dict_alcampo["Name"].append(product.find("span").get_text(strip=True))
                dict_alcampo["Image"].append(product.find("img")["src"])

                dict_alcampo["Category"].append(categoria)

            for price in prices:
                try:
                    dict_alcampo["Price"].append(price.find_all("span")[2].get_text(strip=True))
                except:
                    print("List index out of range")
                    dict_alcampo["Price"].append("NaN")

            for stock in stock_available:
                
                stock_yes_no = stock.find_all("button")[2].get_text(strip=True).split(" "[0])
                
                if stock_yes_no[0]=="Sin":
                    dict_alcampo["Stock"].append("No")
                else:
                    dict_alcampo["Stock"].append("Yes")
                              
            print( "Category:", len(dict_alcampo["Category"]), "Name:", len(dict_alcampo["Name"]) ,"Image:",len(dict_alcampo["Image"]),"Price:", len(dict_alcampo["Price"]), "Stock",len(dict_alcampo["Stock"])    )
            
        contador+=1

df_alcampo = pd.DataFrame(dict_alcampo)

df_alcampo


Leche Huevos Yogures y Lácteos https://www.alcampo.es/compra-online/alimentacion/leche-huevos-yogures-y-lacteos/c/W16?q=%3Arelevance&page=0
Category: 48 Name: 48 Image: 48 Price: 48 Stock 48
Leche Huevos Yogures y Lácteos https://www.alcampo.es/compra-online/alimentacion/leche-huevos-yogures-y-lacteos/c/W16?q=%3Arelevance&page=1
Category: 96 Name: 96 Image: 96 Price: 96 Stock 96
Leche Huevos Yogures y Lácteos https://www.alcampo.es/compra-online/alimentacion/leche-huevos-yogures-y-lacteos/c/W16?q=%3Arelevance&page=2
Category: 144 Name: 144 Image: 144 Price: 144 Stock 144
Leche Huevos Yogures y Lácteos https://www.alcampo.es/compra-online/alimentacion/leche-huevos-yogures-y-lacteos/c/W16?q=%3Arelevance&page=3
Category: 192 Name: 192 Image: 192 Price: 192 Stock 192
Leche Huevos Yogures y Lácteos https://www.alcampo.es/compra-online/alimentacion/leche-huevos-yogures-y-lacteos/c/W16?q=%3Arelevance&page=4
Category: 240 Name: 240 Image: 240 Price: 240 Stock 240
Leche Huevos Yogures y Lácteos

KeyboardInterrupt: 